In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

import pandas as pd

In [ ]:
bank = 'Среднерусский банк'

#### Загружаем файл со всей необходимой информацией

In [ ]:
df = pd.read_excel('URL.xlsx', sheet_name='Ссылки')

#### Сайт даёт провести только 49 скачек в течении 6 часов, следовательно, надо проверить количество ссылок, для которых нужно сформировать QR-код

In [ ]:
df[df['Территориальный банк']==bank].count()

#### Продолжаем

In [ ]:
df = df[df['Территориальный банк']==bank][['ВСП', 'ссылка_текст (после внедрения)']].copy()

df['name'] = df['ВСП'].apply(lambda x: x.replace('/', '-')) #требовалось для последующего переименовывания файлов

df.rename(columns={'ссылка_текст (после внедрения)':'urls'}, inplace=True)

#### Запускаем драйвер

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--kiosk")
driver = webdriver.Chrome('<путь до драйвера>/chromedriver', chrome_options=options)

#### Сохраняем сессию, чтобы подключиться к ней позже

In [ ]:
url = driver.command_executor._url
session_id = driver.session_id

#### Создаём dummy-подключение и закрываем его. Это позволит не закрывать браузер с основным подкючением, что даёт возможность провести все операции, которые не удалось или было слишком наладно оптимизировать (в моём случае это была загрузка логотипа (невозможно автоматизировать на MacOS) и выбор стилей (бот иногда выводил ошибку, если сайт не успевал пролистываться до этого меню, для стабильности я принял решение сделать это один раз вручную))

In [ ]:
driver = webdriver.Remote(command_executor=url,desired_capabilities={})
driver.close()
driver.session_id = session_id

#### Снова подключаемся к основному драйверу

In [ ]:
driver.get("https://www.qrcode-monkey.com")

#### Находим все требуемые для взаимодействия элементы на сайте

In [ ]:
main_button = driver.find_element_by_link_text("URL")
menu_buttons = driver.find_elements_by_class_name("icon")
buttons = driver.find_elements_by_tag_name("button")

#### Осуществляем следующие манипуляции: 
##### 1. Находим поле для заполнения URL
##### 1.1. Очищаем его от дефолтного значения
##### 1.2. Добаляем требуемый URL
##### 2. Ждём одну секунду
##### 2.1. Нажимаем на кнопку "сформировать QR-код"
##### 2.2. Ждём 4 секунды, чтобы код успел сформироваться
##### 2.3. Нажимаем на кнопку "загрузить QR-код"
##### 2.4. Ждём 15 секунд, так как скачка начинается не сразу
##### 3. Закрываем рекламный баннер
##### 3.1 Ждём одну секунду
##### 4. Возвращаемся к началу цикла

In [ ]:
for sber_url in df.urls[:4]:

    field1 = driver.find_element_by_id('qrcodeUrl')
    field1.click()
    field1.clear()
    field1.send_keys(sber_url)

    time.sleep(1)

    buttons[27].click() 

    time.sleep(4)

    buttons[28].click()

    time.sleep(15)

    driver.find_elements_by_xpath("//span[@class='fa fa-times']")[0].click()
    
    time.sleep(1)

In [ ]:
driver.close()